 # Regression Week2: Multiple Regression(Interpretation)

The goal of the notebook is to explore mulitple regression and feature engineering using available graphlab methods.
In this notebook we will predcit the house prices of the given dataset using the multiple regression method.We will:
* Use Sframe to do feature engineering
* Use built in grpahlab functions to compute the regression weights (coefficients/parameters)
* Given regression weights, predictors and outcome write a function to compute the Residual Sum Of Squares(RSS)
* Look at the coefficients and interpret the meanings
* Evaluate multiple models via RSS

# Fire up Graphlab

In [1]:
import graphlab

# Load in house sales data

Dataset is of the house sales data of the King County region

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to prashantbhat94@gmail.com and will expire on February 13, 2020.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\user\AppData\Local\Temp\graphlab_server_1551544892.log.0


# Split data into training and testing.

In [3]:
train_data,test_data = sales.random_split(.8,seed=0)

# Learning a multiple regression model

We will create list of example features we need to predict house price
['sqft_living','#bathrooms','#bedrooms]

In [4]:
example_features = ['sqft_living','bedrooms','bathrooms']
example_model = graphlab.linear_regression.create(train_data,target = 'price', features = example_features, validation_set= None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 3

Number of unpacked features : 3

Number of coefficients    : 4

Starting Newton Method

--------------------------------------------------------

+-----------+----------+--------------+--------------------+---------------+

In [5]:
example_weight_summary = example_model.get('coefficients')
print (example_weight_summary)

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 1.037940     | 4146407.600631     | 258679.804477 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | 87910.0724924  |  7873.3381434 |
| sqft_living |  None | 315.403440552  | 3.45570032585 |
|   bedrooms  |  None | -65080.2155528 | 2717.45685442 |
|  bathrooms  |  None | 6944.02019265  | 3923.11493144 |
+-------------+-------+----------------+---------------+
[4 rows x 4 columns]



# Making Predictions

In [6]:
example_predictions = example_model.predict(train_data)
print(example_predictions[0])

271789.505878


# Compute RSS

In [7]:
def get_residual_sum_of_squares(model,data,outcome):
    #First get the predictions
    predictions = model.predict(data)
    
    #Compute the residual som of squares/errors
    residual = outcome - predictions
    
    #Then square and sum up
    RSS = (residual * residual).sum()
    
    return (RSS)

Test the above function on test data

In [8]:
rss_train_example = get_residual_sum_of_squares(example_model,test_data,test_data['price'])
print(rss_train_example)

2.7376153833e+14


# Create some new features

Now we create some new features by transforming the exisintg features like (sqft_living, #bedrooms, #bathrooms). The transformation can be like log(sqft_living) or multiplication of #bedrooms and #bathrooms

In [9]:
from math import log

Next create following 4 features as columns in TRAIN and TEST:
* bedroom_sqaured = bedrooms * bedrooms
* bed_bath_rooms = bedrooms * bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long

In [10]:
train_data['bedrooms_squared'] = train_data['bedrooms'] * train_data['bedrooms']
test_data['bedrooms_squared'] = test_data['bedrooms'] * test_data['bedrooms']

In [11]:
train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']

train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x:log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x:log(x))

train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

Less than 4 successfully started. Using only 3 workers.

All operations will proceed as normal, but lambda operations will not be able to use all available cores.

(The location of the log file is printed at the start of the GraphLab server).

* Squaring bedrooms will increase the separation between not many bedrooms (eg : 1 1^2 = 1 4 4^2 = 16). Consequently the feature will affect houses with large number of bedrooms.
* bedrooms times bathrooms gives us an "interaction" features. It will be large when both of them are large.
* Taking log of sqaurefeet has the affect of bringing large values closer together
* LAT+LON is time pass lae tus see why

** Quiz Question: What is the mean value of the four new features on the TEST data? (round to 2 digits) **

In [12]:
print(test_data['bedrooms_squared'].mean())
print(test_data['bed_bath_rooms'].mean())
print(test_data['log_sqft_living'].mean())
print(test_data['lat_plus_long'].mean())

12.4466777016
7.50390163159
7.55027467965
-74.6533349722


# Learning Mulitple Models

* Model 1: sqaurefeet, #bedrooms, #bathrooms, lat, long
* Model 2: model 1 + bed_bath_rooms
* Model 3: model 2 + log_sqft_living, bedrooms_squared,lat_plus_long

In [13]:
model1_features = ['sqft_living','bedrooms','bathrooms','lat','long']
model2_features = model1_features + ['bed_bath_rooms']
model3_features = model2_features + ['bedrooms_squared','log_sqft_living','lat_plus_long']

In [14]:
#Learn the 3 models 
model_1 = graphlab.linear_regression.create(train_data,target = 'price', features = model1_features, validation_set=None)
model_2 = graphlab.linear_regression.create(train_data,target = 'price', features = model2_features, validation_set=None)
model_3 = graphlab.linear_regression.create(train_data,target = 'price', features = model3_features, validation_set=None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 5

Number of unpacked features : 5

Number of coefficients    : 6

Starting Newton Method

--------------------------------------------------------

+-----------+----------+--------------+--------------------+---------------+

In [15]:
#Examine/Extract models coeffecients
model_1_coeffecients = model_1.get('coefficients')
model_2_coeffecients = model_2.get('coefficients')
model_3_coeffecients = model_3.get('coefficients')
print(model_1_coeffecients)
print(model_2_coeffecients)
print(model_3_coeffecients)

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.030020     | 4074878.213096     | 236378.596455 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

...

+-----------+----------+--------------+--------------------+---------------+

+------------------+-------+----------------+---------------+
|       name       | index |     value      |     stderr    |
+------------------+-------+----------------+---------------+
|   (intercept)    |  None | -52974974.0602 |  1615194.9439 |
|   sqft_living    |  None | 529.196420564  | 7.69913498511 |
|     bedrooms     |  None | 28948.5277313  | 9395.72889106 |
|    bathrooms     |  None |  65661.207231  | 10795.3380703 |
|       lat        |  None | 704762.148408  | 1292011141.66 |
|       long       |  None | -137780.01994  | 1292011141.57 |
|  bed_bath_rooms  |  None | -8478.36410518 | 2858.95391257 |
| bedrooms_squared |  None | -6072.38466067 | 1494.97042777 |
| log_sqft_living  |  None | -563467.784269 | 17567.8230814 |
|  lat_plus_long   |  None | -83217.1979248 | 1292011141.58 |
+------------------+-------+----------------+---------------+
[10 rows x 4 columns]



| 1         | 2        | 0.027024     | 3193229.177894     | 228200.043155 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

 **Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model1?**
 **Answer: positive**

**Quiz Question: What is the sign(positive or negative) for the coefficint/weight for 'bathrooms' in model2?**
**Answer: negative**

The interpretation of the model1 is 
* house price increase by 310 when the sqft_living variable goes up by 1
* house price decrease by 59577 when the #bedrooms variable goes up by 1
* house price increase by 13811 when the #bathrooms varible goes up by 1
* house price increase by 629865 when the lat variable goes up by 1
* house price decrease by 214790 when the long varibale gos up by 1

The interpretation of the model2 is
* house price increase by 304 when the sqft_living variable increase by 1
* house price decrease by 116366 when the #bedrooms variable increase by 1
* house price decrease by 77972 when the #bathrooms variable increase by 1
* house price increase by 625433 when the lat variable increase by 1
* house price decrease by 203958 when the long variable increase by 1
* hosue price increase by 26961 when the bed_bath_variable increase by 1

# Comparing multiple models

In [16]:
# Compute the RSS on training data for each of the three models and record the values :
res_model_1_train = get_residual_sum_of_squares(model_1, train_data, train_data['price'])
res_model_2_train = get_residual_sum_of_squares(model_2, train_data, train_data['price'])
res_model_3_train = get_residual_sum_of_squares(model_3, train_data, train_data['price'])

print(res_model_1_train)
print(res_model_2_train)
print(res_model_3_train)

9.71328233544e+14
9.61592067856e+14
9.05276314555e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [17]:
# Compute the RSS on TESTING data for each of the three models and record the values:
res_model_1_test = get_residual_sum_of_squares(model_1, test_data, test_data['price'])
res_model_2_test = get_residual_sum_of_squares(model_2, test_data, test_data['price'])
res_model_3_test = get_residual_sum_of_squares(model_3, test_data, test_data['price'])

print(res_model_1_test)
print(res_model_2_test)
print(res_model_3_test)

2.26568089093e+14
2.24368799994e+14
2.51829318952e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.